In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
plt.style.use("seaborn-v0_8")

class IterativeBase():
    def __init__(self,symbol,start,end,amount,use_spread = True):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.initial_balance = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
        self.position = 0
        self.use_spread = use_spread
        #self.get_data()

    def get_data(self):
        raw = pd.read_csv("detailed.csv",parse_dates = ["time"],index_col = "time").dropna()
        raw = raw.loc[self.start:self.end].copy()
        raw["returns"] = np.log(raw["price"]/raw["price"].shift(1))
        self.data = raw

    def plot_data(self,cols=None):
        if cols is None:
            cols = "price"
        self.data[cols].plot(figsize = (12,8),title = self.symbol)

    def get_values(self,bar):
        date = str(self.data.index[bar].date())
        price = round(self.data.price.iloc[bar],5)
        spread = round(self.data.spread.iloc[bar],5)
        return date,price,spread

    def print_current_balance(self,bar):
        date,price,spread = self.get_values(bar)
        print(f"{date} | Current Balance : {round(self.current_balance,2)}")

    def buy_instrument(self,bar,units=None,amount=None):
        date,price,spread = self.get_values(bar)
        if self.use_spread is True:
            price += spread/2
        if amount is not None:
            units = int(amount/price)
        self.current_balance -= units*price
        self.units += units
        self.trades += 1
        print(f"{date} | Buying {units} at {price}")

    def sell_instrument(self,bar,units=None,amount=None):
        date,price,spread = self.get_values(bar)
        if self.use_spread is True:
            price -= spread/2
        if amount is not None:
            units = int(amount/price)
        self.current_balance += units*price
        self.units -= units
        self.trades += 1
        print(f"{date} | Selling {units} at {price}")

    def print_current_position_value(self,bar):
        date,price,spread = self.get_values(bar)
        cpv = self.units*price
        print(f"{date} | Current Position Value : {round(cpv,2)}")
            
    def print_current_nav(self,bar):
        date,price,spread = self.get_values(bar)
        nav = self.units*price + self.current_balance
        print(f"{date} | Net Asset Value : {round(nav,2)}")

    def close_position(self,bar):
        date,price,spread = self.get_values(bar)
        self.current_balance += self.units*price
        self.current_balance -= (abs(self.units)*(spread/2)*self.use_spread)
        print(75*"-")
        print(f"{date} | +++ Closing Final Position +++")
        print(f"{date} | Closing position for {self.units} at {price}")
        self.units = 0
        self.trades += 1
        perf = ((self.current_balance-self.initial_balance)/self.initial_balance) * 100
        self.print_current_balance(bar)
        print(f"{date} | Net Performance = {round(perf,2)}%")
        print(f"{date} | Number of trades executed = {self.trades}")        
        print(75*"-")
        

In [2]:
class IterativeBacktest(IterativeBase):

    #helper method
    def go_long(self, bar, units = None, amount = None):
        if self.position == -1:
            self.buy_instrument(bar, units = -self.units)
        if units:
            self.buy_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount = amount)

    def go_short(self,bar,units = None, amount = None):
        if self.position == 1:
            self.sell_instrument(bar, units = self.units)
        if units:
            self.sell_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount = amount)


    def test_sma_strategy(self,SMA_S,SMA_L):
        

        # initialisation print out
        print(75*"-")
        print(f"Testing SMA strategy | {self.symbol} | SMA_S = {SMA_S} & SMA_L = {SMA_L}")
        print(75*"-")

        # reset
        self.position = 0
        self.trades = 0
        self.current_balance = self.initial_balance
        self.get_data()

        # prepare data
        self.data["SMA_S"] = self.data.rolling(SMA_S).mean()
        self.data["SMA_L"] = self.data.rolling(SMA_L).mean()    
        self.data.dropna(inplace = True)

        # sma crossover strategy
        for bar in range(len(self.data-1)):
            if self.data["SMA_S"].iloc[bar] > self.data["SMA_L"].iloc[bar]:
                if self.position in [0,-1]:
                    self.go_long(bar,amount = "all")
                    self.position = 1
            elif self.data["SMA_S"].iloc[bar] < self.data["SMA_L"].iloc[bar]:
                if self.position in [0,1]:
                    self.go_short(bar,amount = "all")
                    self.position = -1
        self.close_position(bar+1)

                









In [3]:
bc = IterativeBacktest("EURUSD", "2006-12-31", "2020-06-30", 100000, use_spread= True)